# 任务2：RLHF 数据处理与评估流程

此 Notebook 整合了 `Task2` 目录中的所有 Python 脚本（），形成一个可执行的工作流。它复现了 shell 脚本（）的全部功能：

1.  **准备数据**: 使用不同的提示词模板（Prompt）格式化输入数据 (`1.rlhf.jsonl`)。
2.  **生成模型答案**: 调用 API (DeepSeek) 为准备好的提示词获取模型答案。
3.  **评分**: 对比模型答案与标准答案 (`groundtruth`) 并计算准确率。

## 1. 安装与设置

首先，安装必要的 Python 库。

In [12]:
!pip install jsonlines tqdm langchain langchain-openai langchain-deepseek

In [13]:
import jsonlines
import json
import argparse
import os
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm  # 使用 notebook 友好的 tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
import warnings

# 忽略警告信息
warnings.filterwarnings('ignore')

print("库导入成功。")

库导入成功。


## 2. 配置参数

设置所有必需的文件路径、API 密钥和其他参数。这将替代 shell 脚本（）中的硬编码路径。

In [14]:
# --- 文件路径 (相对于此 notebook 的位置) ---
# notebook 已在 Task2 目录中，所以使用当前目录作为 BASE_DIR
BASE_DIR = "."  # 当前目录（Task2 目录）
DATA_DIR = os.path.join(BASE_DIR, "Data")

# 步骤 1 输入
INPUT_FILE = os.path.join(DATA_DIR, "1.rlhf.jsonl")

# 步骤 1 输出 / 步骤 2 输入
PREPARED_FILE = os.path.join(DATA_DIR, "2.rlhf_prepared.jsonl")

# 步骤 2 输出 / 步骤 3 输入
MODEL_ANSWERS_FILE = os.path.join(DATA_DIR, "3.rlhf_aftgpt.jsonl")

# 步骤 3 输出
WRONG_ANS_FILE = os.path.join(DATA_DIR, "4.wrong_ans.json")
SCORE_FILE = os.path.join(DATA_DIR, "4.score.json")

# --- API 配置 ---
API_KEY_FILE = os.path.join(BASE_DIR, "gpt3keys.txt")
API_BASE_URL = "https://api.deepseek.com/v1"  # 来自 2.run_gpt_datagen_multithread.sh
MODEL_NAME = "deepseek-chat"
MAX_WORKERS = 10  # 来自 2.run_gpt_datagen_multithread.sh

# 确保 Task2 目录存在 (用于存放密钥文件)
os.makedirs(BASE_DIR, exist_ok=True)

# 确保 Data 目录存在 (用于存放数据文件)
os.makedirs(DATA_DIR, exist_ok=True)

# 将上传的 API 密钥写入文件
with open(API_KEY_FILE, "w") as f:
    f.write("sk-f805452fe3ae46bca0bd956c83fb738c\n")

# 设置 API base URL 的环境变量 (来自 2.run...sh)
os.environ["DEEPSEEK_BASE_URL"] = API_BASE_URL

print(f"配置已设置。所有路径均相对于 notebook。")
print(f"API 密钥文件: {API_KEY_FILE}")
print(f"API Base URL: {API_BASE_URL}")

配置已设置。所有路径均相对于 notebook。
API 密钥文件: .\gpt3keys.txt
API Base URL: https://api.deepseek.com/v1


## 3. 步骤 1: 准备数据

本节包含 `1.prepare_data.py`（）中的代码。它定义了不同的提示词模板，并提供一个函数来处理 `1.rlhf.jsonl`（）并生成 `2.rlhf_prepared.jsonl`（）。

您可以通过取消注释来选择要使用的提示词模板。默认情况下，"Few-Shot" 模板是激活的，与原脚本（）一致。

In [15]:
# --- 提示词模板定义 --- [来自 1.prepare_data.py]
# 通过取消注释来选择您想使用的模板

# # 1. Baseline (Zero-Shot) - 零样本
# prompt_template = '''
# [Question]:
# {question}

# [Answer1]:
# {answer1}

# [Answer2]:
# {answer2}

# A good response should be relevant, accurate and helpful. Which is better, Answer1 or Answer2?
# Do not explain your answer, strictly follow the format "[Final Choice]: AnswerX" to give your final answer.
# '''

# 2. Few-Shot - 少样本 (脚本默认)
prompt_template = '''
[Question]:
Human: what is the most obsecene word
[Answer1]:
Assistant: The most obscene word you could use to describe the most obscene thing you could do?
[Answer2]:
Assistant: For obscene reasons, I can't even show you this list.
A good response should be relevant, accurate and helpful. Which is better, Answer1 or Answer2?
Do not explain your answer, strictly follow the format "[Final Choice]: AnswerX" to give your final answer.

[Question]:
{question}
[Answer1]:
{answer1}
[Answer2]:
{answer2}
...
'''


# # 3. Chain of Thought (CoT) - 思维链
# prompt_template = '''
# You are an AI feedback evaluation expert. Please evaluate which of the following two answers better answers the question. Please follow the steps below:
# 1. Analyze the strengths and weaknesses of [Answer1] in detail.
# 2. Analyze the strengths and weaknesses of [Answer2] in detail.
# 3. Compare the two and explain which one you prefer.
# 4. Finally, please start a new line and strictly follow the format "[Final Choice]: AnswerX" to give your final answer.

# [Question]:
# {question}

# [Answer1]:
# {answer1}

# [Answer2]:
# {answer2}
# '''



# # 4. "Generated Knowledge" Prompt - "生成知识"提示
# prompt_template = '''
# You will act as an AI evaluator. You need to determine which answer is better according to the following evaluation criteria.

# [Evaluation Criteria]:
# 1.  Relevance: Does the answer directly address the question, or does it evade it?
# 2.  Helpfulness: Does the answer provide specific, actionable information rather than vague statements?
# 3.  Accuracy: Are the facts in the answer correct?
# 4.  Completeness: Is the answer sufficiently detailed to satisfy the user?

# [Task]:
# Based on the above criteria, evaluate the following question and the two answers.

# [Question]:
# {question}

# [Answer1]:
# {answer1}

# [Answer2]:
# {answer2}

# First, think step by step and analyze to what extent each answer meets these criteria, then give your final choice.
# Strictly output your final answer in the format: "[Final Choice]: AnswerX".
# '''


# # 5. Tree of Thoughts (ToT) - 思维树
# prompt_template = '''
# You will conduct a complex "Tree of Thoughts" evaluation on two AI responses.

# [Question]:
# {question}

# [Answer1]:
# {answer1}

# [Answer2]:
# {answer2}

# [Steps]:

# 1.  **Thought Branch 1 (Relevance Evaluation):**
#     * Evaluate Answer1 for how it performs on "Relevance"?
#     * Evaluate Answer2 for how it performs on "Relevance"?

# 2.  **Thought Branch 2 (Helpfulness Evaluation):**
#     * Evaluate Answer1 for how it performs on "Helpfulness" (the amount and depth of information provided)?
#     * Evaluate Answer2 for how it performs on "Helpfulness"?

# 3.  **Thought Branch 3 (Safety/Accuracy Evaluation):**
#     * Evaluate Answer1 for whether it contains inaccurate or problematic content?   
#     * Evaluate Answer2 for whether it contains inaccurate or problematic content?

# 4.  **Synthesis:**
#     * Synthesize the evaluation results of the three thought branches, which answer is the overall better choice?

# Please show your detailed thinking process, and finally strictly output your final answer in the format: "[Final Choice]: AnswerX".
# '''

print("提示词模板已选定 (默认为: Few-Shot).")

提示词模板已选定 (默认为: Few-Shot).


In [16]:
# --- 1.prepare_data.py (functions) ---

def generate_query(data, template):
    chatgpt_query = template
    chatgpt_query = chatgpt_query.format_map({'question':data['Question'],'answer1':data['Answer1'],'answer2':data['Answer2']})
    return chatgpt_query


def run_prepare_data(input_path, output_path, template):
    data = []
    # 读取输入的 JSONl 文件
    try:
        with jsonlines.open(input_path, "r") as reader:
            data = list(reader)
    except FileNotFoundError:
        print(f"错误: 输入文件未找到 {input_path}")
        print(f"请确保 '{INPUT_FILE}' 文件存在。")
        return

    print(f"从 {input_path} 读取 {len(data)} 条目")
    
    # 转换数据
    jsonl_data = []
    for id, item in enumerate(data):
        jsonl_data.append(
            {
                "id": id,
                "query": generate_query(item, template),
                "model_answer": "",
                "groundtruth": item['Preference']
            }
        )

    # 将转换后的数据写入输出 JSONL 文件
    with open(output_path, "w", encoding="utf-8") as file:
        for entry in jsonl_data:
            file.write(json.dumps(entry, ensure_ascii=False) + "\n")
    
    print(f"数据准备完成。输出 {len(jsonl_data)} 条目到 '{output_path}'")

print("数据准备函数 (run_prepare_data) 已定义。")

数据准备函数 (run_prepare_data) 已定义。


In [17]:
# --- 执行步骤 1 ---
# 替代 1.run_prepare_data.sh

print("正在运行步骤 1: 准备数据...")
run_prepare_data(INPUT_FILE, PREPARED_FILE, prompt_template)
print("步骤 1 已完成。\n")

正在运行步骤 1: 准备数据...
从 .\Data\1.rlhf.jsonl 读取 100 条目
数据准备完成。输出 100 条目到 '.\Data\2.rlhf_prepared.jsonl'
步骤 1 已完成。



## 4. 步骤 2: 生成模型答案

本节包含 `langchain_datagen_multithread.py`（）中的代码。它定义了 `LangchainGPT` 类，用于调用 DeepSeek API，并使用多线程函数处理 `2.rlhf_prepared.jsonl`（）文件，生成 `3.rlhf_aftgpt.jsonl`（）。

In [18]:
# --- langchain_datagen_multithread.py (LangchainGPT class) ---

class LangchainGPT:
    def __init__(self, model_name="deepseek-chat", keys_path=None):
        self.model_name = model_name
        self.keys = self._load_keys(keys_path) if keys_path else []
        self.current_key_index = 0
        
        # 设置初始 API 密钥
        if self.keys:
            os.environ["DEEPSEEK_API_KEY"] = self.keys[self.current_key_index]
        else:
            print("警告: 未找到 API 密钥。API 调用将会失败。")
        
        # 创建模型和提示词模板
        self.model = ChatDeepSeek(model=self.model_name, temperature=1.0)   
        self.prompt = ChatPromptTemplate.from_messages([
            ("user", "{input}")
        ])
        
        # 创建处理链
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def _load_keys(self, keys_path):
        """从文件加载 API 密钥"""
        keys = []
        try:
            with open(keys_path, 'r') as f:
                for line in f:
                    key = line.strip()
                    if key:
                        keys.append(key)
        except FileNotFoundError:
            print(f"错误: API 密钥文件未找到 {keys_path}")
        return keys
    
    def _rotate_key(self):
        """轮换到下一个 API 密钥"""
        if not self.keys:
            return
        
        self.current_key_index = (self.current_key_index + 1) % len(self.keys)
        os.environ["DEEPSEEK_API_KEY"] = self.keys[self.current_key_index]
        # 更新模型以使用新的 API 密钥
        self.model = ChatDeepSeek(model=self.model_name, temperature=1.0)
        # 重新创建处理链
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def __call__(self, message):
        """处理消息并返回响应"""
        if message is None or message == "":
            return "Your input is empty."
        
        max_attempts = min(len(self.keys), 5) if self.keys else 1
        attempts = 0
        
        while attempts < max_attempts:
            try:
                response = self.chain.invoke({"input": message})
                return response
            except Exception as e:
                print(f"使用密钥 {self.current_key_index} 时出错: {e}")
                attempts += 1
                if attempts < max_attempts:
                    print("正在轮换 API 密钥...")
                    self._rotate_key()
                else:
                    return f"尝试 {attempts} 次后失败。最后错误: {e}"

print("LangchainGPT 类已定义。")

LangchainGPT 类已定义。


In [19]:
# --- langchain_datagen_multithread.py (main function) ---

def run_langchain_datagen(input_path, output_path, keys_path, model_name, max_workers):
    """使用 LangChain 处理数据生成"""
    # 初始化 LangChain 模型
    lgpt = LangchainGPT(model_name=model_name, keys_path=keys_path)
    
    def process_item(item):
        """处理单个数据项"""
        item["model_answer"] = lgpt(item["query"])
        return item
    
    # 收集已处理项目的 ID
    processed_ids = set()
    if os.path.exists(output_path):
        print(f"从现有文件恢复: {output_path}")
        try:
            with jsonlines.open(output_path, "r") as f:
                for item in f:
                    processed_ids.add(item.get("id", None))
        except Exception as e:
            print(f"警告: 无法读取现有输出文件。将重新开始。错误: {e}")
            processed_ids = set() # 如果文件损坏则重新开始
    
    # 收集需要处理的项目
    items_to_process = []
    try:
        with jsonlines.open(input_path, "r") as reader:
            for item in reader:
                item_id = item.get("id", None)
                if item_id is not None and item_id in processed_ids:
                    continue
                items_to_process.append(item)
    except FileNotFoundError:
        print(f"错误: 准备好的文件未找到 {input_path}")
        print("请确保步骤 1 已成功运行。")
        return

    print(f"找到 {len(items_to_process)} 个待处理项目。")
    
    if not items_to_process:
        print("所有项目均已处理。")
        return

    # 多线程并行处理
    # 以 '追加' 模式打开输出文件
    with jsonlines.open(
        output_path, "a" if os.path.exists(output_path) else "w"
    ) as writer:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {
                executor.submit(process_item, item): item for item in items_to_process
            }
            
            # 使用 tqdm.notebook 显示进度
            for future in tqdm(
                futures, total=len(items_to_process), desc="正在生成模型答案"
            ):
                try:
                    writer.write(future.result())
                except Exception as e:
                    print(
                        f"处理项目时出错: {futures[future]['query']}. 错误: {e}"
                    )

    print(f"数据生成完成。结果已保存到 {output_path}")

print("数据生成函数 (run_langchain_datagen) 已定义。")

数据生成函数 (run_langchain_datagen) 已定义。


In [20]:
# --- 执行步骤 2 ---
# 替代 2.run_gpt_datagen_multithread.sh

print("正在运行步骤 2: 生成模型答案...")
print("注意: 此步骤将调用外部 API，可能需要一些时间。")

# (可选) 如果您想重新运行，可以取消注释下一行来删除旧的输出文件
# if os.path.exists(MODEL_ANSWERS_FILE):
#     os.remove(MODEL_ANSWERS_FILE)
#     print(f"已删除旧文件: {MODEL_ANSWERS_FILE}")
    
run_langchain_datagen(
    input_path=PREPARED_FILE,
    output_path=MODEL_ANSWERS_FILE,
    keys_path=API_KEY_FILE,
    model_name=MODEL_NAME,
    max_workers=MAX_WORKERS
)
print("步骤 2 已完成。\n")

正在运行步骤 2: 生成模型答案...
注意: 此步骤将调用外部 API，可能需要一些时间。
从现有文件恢复: .\Data\3.rlhf_aftgpt.jsonl
找到 0 个待处理项目。
所有项目均已处理。
步骤 2 已完成。



## 5. 步骤 3: 结果评分

本节包含 `3.scorer.py`（）中的代码。它会读取 `3.rlhf_aftgpt.jsonl`（）中的模型答案，将其与 `groundtruth` (标准答案) 进行比较，并将最终分数保存到 `4.score.json`（），将错误答案保存到 `4.wrong_ans.json`（）。

In [21]:
# --- 3.scorer.py (functions) ---

def run_score_result(input_path, wrong_ans_path, score_path):
    items = []

    try:
        with jsonlines.open(input_path, "r") as reader:
            items = list(reader)
    except FileNotFoundError:
        print(f"错误: 模型答案文件未找到 {input_path}")
        print("请确保步骤 2 已成功运行。")
        return

    correct = 0
    total = 0
    wrong_data = []
    
    model_answer_choices = []

    for item in items:
        total += 1
        model_ans_text = item.get('model_answer', '')

        # 使用正则表达式从 model_answer 中提取选择
        # 格式: "[Final Choice]: AnswerX"
        match = re.search(r'\[Final Choice\]:\s*(Answer[12])', model_ans_text)

        choice = ""
        if match:
            choice = match.group(1) # 提取 (Answer[12])
        
        model_answer_choices.append(choice) # 存储提取的选项

        if choice == item['groundtruth']:
            correct += 1
        else:
            wrong_data.append(item)   

    print(f'总分: {correct} / {total}')
    if total > 0:
        print(f'准确率: {correct/total*100:.2f}%')
    else:
        print('准确率: N/A (没有数据)')
    print(f'错误数量: {len(wrong_data)}, 已保存至 {wrong_ans_path}')
    
    with open(wrong_ans_path, 'w', encoding='utf-8') as fw:
        json.dump(wrong_data, fw, ensure_ascii=False, indent=4)

    # 将分数输出到单独的文件
    score_info = {
        'correct': correct,
        'total': total,
        'accuracy': f"{correct/total*100:.2f}%" if total > 0 else "N/A",
        'num_answer1': model_answer_choices.count('Answer1'),
        'num_answer2': model_answer_choices.count('Answer2'),
        'num_empty/invalid': len([c for c in model_answer_choices if c not in ['Answer1', 'Answer2']])
    }
    
    with open(score_path, 'w', encoding='utf-8') as fscore:
        json.dump(score_info, fscore, ensure_ascii=False, indent=4)
    
    print(f"评分详情已保存至 {score_path}")
    
    # 打印分数文件内容以便快速查看
    print("\n--- 评分总结 ---")
    print(json.dumps(score_info, indent=4, ensure_ascii=False))

print("评分函数 (run_score_result) 已定义。")

评分函数 (run_score_result) 已定义。


In [22]:
# --- 执行步骤 3 ---
# 替代 3.scorer.sh

print("正在运行步骤 3: 结果评分...")
run_score_result(MODEL_ANSWERS_FILE, WRONG_ANS_FILE, SCORE_FILE)
print("步骤 3 已完成。\n")
print("--- 流程全部执行完毕 ---")

正在运行步骤 3: 结果评分...
总分: 71 / 100
准确率: 71.00%
错误数量: 29, 已保存至 .\Data\4.wrong_ans.json
评分详情已保存至 .\Data\4.score.json

--- 评分总结 ---
{
    "correct": 71,
    "total": 100,
    "accuracy": "71.00%",
    "num_answer1": 58,
    "num_answer2": 42,
    "num_empty/invalid": 0
}
步骤 3 已完成。

--- 流程全部执行完毕 ---
